In [2]:
import pandas as pd
import numpy as np

In [3]:
fifa = pd.read_csv("../Data/fifa22.csv",low_memory=False)
#Change all positions into either G=Gaolskeeper, D=Defence, M=Midfield, A=Attacking
posDict={'GK':'GK', 'LW':"A", 'CAM':'A', 'RW':'A', 'CM':'M', 'RWB':'D', 'CDM':'M', 'LM':'M',
         'CF':'A', 'ST':'A', 'LB':'D', 'CB':'D', 'RB':'D', 'LWB':'D', 'RM':'M'}
#Apply to data
pos=list(fifa['player_positions'])
fifa['position']=fifa['player_positions']
for i in range(len(pos)):
    fifa['position'][i]=posDict[(pos[i].split(', '))[0]] #Take first value
    
fifa=fifa.drop(columns=['player_positions'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [5]:
fifa.to_csv("../Data/autoFifa22.csv")

In [6]:
import boto3
s3 = boto3.client('s3')

def uploadDataS3(data, bucket, key):
    try:
        s3.upload_file(data, bucket, key)
        print("Upload Successful")
        return True
    except: 
        print("Upload Failed")
    
uploaded = uploadDataS3('../Data/autoFifa22.csv', 'sagemaker-us-east-1-022761975017', 'autoML/autoFifa22.csv')

Upload Successful


In [47]:
### Lets use our best model to make predictions
import boto3, sys, time

ep_name = 'fifaAutoMl-Endpoint-2022-04-12'
sm_rt = boto3.Session().client('runtime.sagemaker')

start_time = time.time()
predictions = []
labels = []
with open('../Data/autoFifa22.csv') as f:
    lines = f.readlines()
    for l in lines[1:]:   # Skip header
        l = l.split(',')      # Split CSV line into features
        label = l[-1]         # Store 'position' label
        l = l[:-1]            # Remove label
        l = ','.join(l)       # Rebuild CSV line without label
                
        response = sm_rt.invoke_endpoint(EndpointName=ep_name, 
                                         ContentType='text/csv',       
                                         Accept='text/csv', Body=l)

        response = response['Body'].read().decode("utf-8")
        #predictions.append(response)
        cur_predictions = response[0]
        predictions.append(cur_predictions)
        labels.append(label[:-1])
        #print ("label %s response %s" %(label,response))

print ("Done")
run_time = time.time() - start_time
predictions=np.array(predictions)
labels=np.array(labels)
test_size = labels.shape[0]
num_correct = sum(predictions == labels)
accuracy = num_correct / float(test_size)
print("time required for predicting %d data point: %.2f seconds" % (test_size, run_time))
print("accuracy of model: %.1f%%" % (accuracy * 100))               



Done
time required for predicting 19239 data point: 874.93 seconds
accuracy of model: 88.9%


In [48]:
def delete_endpoint(name):
    try:
        boto3.client("sagemaker").delete_endpoint(EndpointName=name)
        print(f"Deleted {name}")
    except:
        print(f"Already deleted: {name}")


delete_endpoint(ep_name)

Deleted fifaAutoMl-Endpoint-2022-04-12
